In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
## load the ANN trained model, scaler, and encoder 
model = load_model('ann_model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
    
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder = pickle.load(f)
    
with open('one_hot_encoder_geo.pkl', 'rb') as f:
    one_hot_encoder_geo = pickle.load(f)

In [4]:
# example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [11]:
# preprocess the input data
def preprocess_input(data):
    # convert to DataFrame
    df = pd.DataFrame([data])

    # encode Gender using label encoder
    df["Gender"] = label_encoder.transform(df["Gender"])

    # one-hot encode Geography
    geo_encoded = one_hot_encoder_geo.transform(df[["Geography"]])
    geo_encoded_df = pd.DataFrame(
        geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(["Geography"])
    )

    # drop original Geography column and add encoded columns
    df = df.drop("Geography", axis=1)
    df = pd.concat([df, geo_encoded_df], axis=1)
    
    # scale the features
    df_scaled = scaler.transform(df)

    return df_scaled


In [12]:
df = preprocess_input(input_data)

In [15]:
df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
# prdiction
predictions = model.predict(df)

1/1 [==============================] - 1s 575ms/step


In [ ]:
predictions

array([[0.05796852]], dtype=float32)

In [ ]:
if predictions[0][0] > 0.5:
    print("Customer is likely to leave the bank.")
else:
    print("Customer is likely to stay with the bank.")